The previous version of GPS datatime converter was written in 2020. I used very simple concept to deal with the problem. However, the basic package was not enough to solve some complicated issues. Therefore, I update the algorithm, using some codes got from Stackoverflow

The output of this version will be saved in `Excel` file format

However, I got some issue related to duplication

The reason because some data points are recorded in the same day (as a compensation for the previous missing data)

So I upgrade the script to support my manual work of modifying the duplication

In [1]:
import datetime
import os
from datetime import datetime

import numpy as np
import pandas as pd

In [2]:
where_to_FIND = r"D:\VINHTRUONG_PHD\INSAR\202__GPS_Data\2__GPS_Sinica_2023\GNSS資料"
where_to_SAVE = r"D:\VINHTRUONG_PHD\INSAR\202__GPS_Data\2__GPS_Sinica_2023\Converted_GNSS_CSVFiles"
# where_to_SAVE = (
#     r"D:\VINHTRUONG_PHD\INSAR\202__GPS_Data\2__GPS_Sinica_2023\Converted_GNSS_ExcelFiles"
# )

if not os.path.exists(where_to_SAVE):
    os.makedirs(where_to_SAVE)

In [3]:
def datetime_converter(frac_year):
    """
    Input:

    frac_year (list) : the input fractional year that need to be converted

    """
    import pandas as pd

    # first, change each value in the input list
    # into year only (dtype: integer)
    # for example, 2007.1234 --> 2007
    to_year = pd.to_datetime(frac_year, format="%Y")

    # we need to determine days in the given year
    # to calculate days from fractional year
    # we might need to consider leap years; if not leap years, just use 365.25
    # use `where` function to apply command by conditions
    days_in_year = pd.Series([366] * to_year.size).where(to_year.dt.is_leap_year, 365.25)

    # now we can add the days as a timedelta
    time_delta = frac_year - frac_year.astype(int)  # take the fraction
    output = to_year + pd.to_timedelta(days_in_year * time_delta, unit="d")

    return output.tolist()

In [4]:
READFOLDER = where_to_FIND
WRITEFOLDER = where_to_SAVE

FILELIST = []
SAVELIST = []

extension = ".neu_rmoffEQ"
for FILENAME in os.listdir(READFOLDER):
    #     if FILENAME.endswith('.txt'):
    if FILENAME.endswith(extension):
        FILEPATH = os.path.join(READFOLDER, FILENAME).replace("\\", "/")
        FILESAVE = os.path.join(WRITEFOLDER, FILENAME).replace("\\", "/")
        FILELIST.append(FILEPATH)
        SAVELIST.append(FILESAVE)

In [5]:
for FILEPATH, SAVEPATH in zip(FILELIST, SAVELIST):
    try:
        DATA = pd.read_table(FILEPATH, header=None, sep="\s+")

        DATA.columns = ["Datetime", "dN", "dE", "dU", "sN", "sE", "sU"]

        RAW_YEAR = DATA["Datetime"]

        CONVERTED = datetime_converter(RAW_YEAR)

        dataframe = pd.DataFrame(data=None, index=None, dtype=None)
        dataframe["Original"] = DATA["Datetime"]
        dataframe["Converted"] = [datetime(a.year, a.month, a.day) for a in CONVERTED]
        dataframe["dN(m)"] = DATA["dN"]
        dataframe["dE(m)"] = DATA["dE"]
        dataframe["dU(m)"] = DATA["dU"]
        dataframe["sN(m)"] = DATA["sN"]
        dataframe["sE(m)"] = DATA["sE"]
        dataframe["sU(m)"] = DATA["sU"]

        SAVENAME = SAVEPATH.replace(extension, ".csv")
        dataframe.to_csv(SAVENAME, header=True, index=False, index_label=None)

    except Exception as e:
        print(os.path.basename(FILEPATH))
        print(e)
        pass